In [ ]:
import sqlite3 as sql
import pandas as pd
import json

# SQLite/Python test

In [ ]:
con = sql.connect(":memory:")
cur = con.cursor()

cur.execute(
    """CREATE TABLE scenarios(\
id INTEGER PRIMARY KEY,
created TEXT NOT NULL,
completed TEXT,
num_reps INTEGER NOT NULL CHECK (num_reps > 0),
reps_done INTEGER NOT NULL CHECK (reps_done >= 0),
sim_hours REAL NOT NULL CHECK (sim_hours > 0),
result TEXT)
"""
)

In [ ]:
cur.execute(
    "INSERT INTO scenarios VALUES(?,?,?,?,?,?,?)",
    (None, sql.Timestamp.utcnow().isoformat(), None, 10, 0, 1008.0, None)
)
scenario_id = cur.lastrowid
con.commit()
print(scenario_id)

In [ ]:
pd.read_sql('SELECT * FROM scenarios', con, parse_dates={'created': {'utc': True}, 'completed': {'utc': True}})

In [ ]:
cur.execute(
    """UPDATE scenarios
    SET `reps_done` = `reps_done` + 1
    WHERE id=?
    """,
    (scenario_id, )
)
con.commit()

pd.read_sql('SELECT * FROM scenarios', con, parse_dates={'created': {'utc': True}, 'completed': {'utc': True}})

In [ ]:
cur.execute(
    """UPDATE scenarios
    SET
        `reps_done` = 10,
        `completed` = ?
    WHERE id=?
    """,
    (sql.Timestamp.utcnow().isoformat(), scenario_id)
)
con.commit()

df = pd.read_sql('SELECT * FROM scenarios', con, parse_dates={'created': {'utc': True}, 'completed': {'utc': True}})
df

In [ ]:
df.dtypes

### JSON support

In [ ]:
data = {'hello': 123, 'world': [4,5,6], 'this': 'is a test', 'the quick': 'brown fox jumps over the lazy dog'.split()}
cur.execute(
    """UPDATE scenarios
    SET
        result = ?
    WHERE id=?
    """,
    (json.dumps(data), scenario_id)
)
con.commit()

df = pd.read_sql('SELECT *, LENGTH(`result`) FROM scenarios', con, parse_dates={'created': {'utc': True}, 'completed': {'utc': True}})
df

In [ ]:
pd.read_sql("""SELECT json_extract(`result`, '$.this', '$.the quick[2]') AS json1 FROM scenarios""", con)